In [ ]:
!unzip Train.zip

Archive:  Train.zip
   creating: Train/
   creating: Train/Test/
   creating: Train/Test/Horses/
  inflating: Train/Test/Horses/horse02-1.png  
  inflating: Train/Test/Horses/horse02-2.png  
  inflating: Train/Test/Horses/horse02-3.png  
  inflating: Train/Test/Horses/horse02-4.png  
  inflating: Train/Test/Horses/horse02-5.png  
  inflating: Train/Test/Horses/horse02-6.png  
  inflating: Train/Test/Horses/horse02-7.png  
  inflating: Train/Test/Horses/horse02-8.png  
   creating: Train/Test/Humans/
  inflating: Train/Test/Humans/human01-11.png  
  inflating: Train/Test/Humans/human01-12.png  
  inflating: Train/Test/Humans/human01-13.png  
  inflating: Train/Test/Humans/human01-14.png  
  inflating: Train/Test/Humans/human01-15.png  
  inflating: Train/Test/Humans/human01-16.png  
  inflating: Train/Test/Humans/human01-17.png  
  inflating: Train/Test/Humans/human01-18.png  
   creating: Train/Train/
   creating: Train/Train/Horses/
  inflating: Train/Train/Horses/horse01-1.png  
  in

In [ ]:
!ls

sample_data  Train  Train.zip


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = '/content/Train/Train'
test_dir = '/content/Train/Test'

In [ ]:

train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=20,zoom_range=0.15,
                                   width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,
                                   horizontal_flip=True,fill_mode='nearest')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 16 images belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 16 images belonging to 2 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=test_generator)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.5625 - loss: 0.6937 - val_accuracy: 0.5000 - val_loss: 0.6925
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6250 - loss: 0.6791 - val_accuracy: 0.4375 - val_loss: 0.6996
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 617ms/step - accuracy: 0.5625 - loss: 0.6909 - val_accuracy: 0.5000 - val_loss: 0.7060
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step - accuracy: 0.6875 - loss: 0.6301 - val_accuracy: 0.5000 - val_loss: 0.7065
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 0.6250 - loss: 0.6777 - val_accuracy: 0.2500 - val_loss: 0.6970
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - accuracy: 0.5000 - loss: 0.6959 - val_accuracy: 0.5000 - val_loss: 0.6971
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.6875 - loss: 0.5575 - val_accuracy: 0.5000 - val_loss: 0.6962
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 0.5625 - loss: 0.6433 - val_accuracy: 0.4375 - val_loss: 0.6965
Epoch 9/2

In [ ]:
model.save('mymodel.keras')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
def predict_image(img_path,model):
  img = image.load_img(img_path,target_size=(150,150))
  img_tensor =image.img_to_array(img)
  img_tensor = np.expand_dims(img_tensor,axis=0)
  img_tensor /=255.
  prediction = model.predict(img_tensor)
  if prediction[0] > 0.5:
    print('The image is a Human !')
  else:
    print('The image is a Horse !')

In [ ]:
predict_image('/content/horse01-7.png',model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
The image is a Horse !


In [ ]:
predict_image('/content/human01-02.png',model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
The image is a Human !
